In [ ]:
import json
import jsonlines
import networkx as nx
import hypernetx as hnx
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
from refined.inference.processor import Refined
from collections import defaultdict


In [ ]:
refined = Refined.from_pretrained(model_name='wikipedia_model_with_numbers',
                                  entity_set="wikipedia")

In [ ]:
def save_json(data, filepath=r'new_data.json'):
    with open(filepath, 'w') as fp:
        json.dump(data, fp, indent=4)

In [ ]:
def merge_sentences(sentences):
    sentence_list = [" ".join(sentence_word_list) for sentence_word_list in sentences] # merge the words into sentences
    paragraph = " ".join(sentence_list)
    return paragraph

def prepare_events(datum):
    return datum['events']
    # words_flattened = [word for sentence in datum['sentences'] for word in sentence]
    for event in datum['events']:
        # trigger = event['trigger']
        arguments = event['arguments']
        arguments_obj = [
            { 
                'argument_id': argument, 
                'argument_word': argument,
            }
            for argument in arguments
            ]
        event['arguments'] = arguments_obj
    return datum['events']

def link_entities(event, paragraph):
    # spans = refined.process_text(paragraph)
    res = []
    for participant in event['participants']:
        spans = refined.process_text(participant)
        if spans == []: 
            participant = {
                "entity_id": participant,
                "entity_title": participant,
                "raw_mention": participant,
                "entity_type": "None"
            }
            res.append(participant)
            continue
        span = [span for span in spans][0]
        entity_word = span.text
        print(span)
        if span.predicted_entity != None and span.predicted_entity.wikidata_entity_id != None:
            participant = {
                "entity_id": span.predicted_entity.wikidata_entity_id,
                "entity_title": span.predicted_entity.wikipedia_entity_title,
                "raw_mention": participant,
                "entity_type": span.coarse_mention_type or "None",
            }
        else:
            participant = {
                "entity_id": participant,
                "entity_title": participant,
                "raw_mention": participant,
                "entity_type": span.coarse_mention_type or "None",
            }
        res.append(participant)
    event['participants'] = res
    return event

    for span in spans:
        entity_word = span.text
        for event in events:
            for argument in event['arguments']:
                if argument['argument_word'] == entity_word:
                    if span.predicted_entity != None and span.predicted_entity.wikidata_entity_id != None:
                        entity_id = span.predicted_entity.wikidata_entity_id
                        entity_title = span.predicted_entity.wikipedia_entity_title
                        argument['argument_id'] = entity_id
                        argument['entity_title'] = entity_title
                    argument['entity_type'] = span.coarse_mention_type 
    return events

def transform_dataset(dataset):
    transformed_dataset = {}
    for index, datum in enumerate(dataset):
        print("{}/{}".format(index, len(dataset)))
        paragraph = datum['summary']
        event = prepare_events(datum)
        event = link_entities(event, paragraph)
        doc_key = datum['id']
        source_url = datum['url']
        if doc_key not in transformed_dataset.keys():
            datum['events'] = event
            transformed_dataset[doc_key] = datum
    return list(transformed_dataset.values())

def remove_duplicates(dataset):
    reverse_index_url = defaultdict(list)
    kept_dataset = []
    for index, datum in enumerate(dataset):
        if datum['source_url'] in reverse_index_url.keys():
            previous_data = reverse_index_url[datum['source_url']]
            duplicate = False
            for previous_datum in previous_data:
                if " ".join(datum['content'][0][0]) == " ".join(previous_datum['content'][0][0]):
                    duplicate = True
                    break
            if duplicate: continue
        reverse_index_url[datum['source_url']].append(datum)
        kept_dataset.append(datum)
    return kept_dataset


In [18]:
import re
def find_participant_mentions(participants, paragraph):
    spans = refined.process_text(paragraph)
    print(spans)
    participant_ids = [participant['entity_id'] for participant in participants]
    mentioned_participants = []
    mention_texts = []
    for span in spans:
        if span.predicted_entity != None and span.predicted_entity.wikidata_entity_id != None:
            entity_id = span.predicted_entity.wikidata_entity_id
            entity_title = span.predicted_entity.wikipedia_entity_title
            mention = span.text
            if entity_id in participant_ids and mention not in mention_texts:
                mention_texts.append(mention)
                mentioned_participants.append({
                    'text': mention,
                    'entity_id': entity_id,
                    'entity_title': entity_title,
                })
    if len(mentioned_participants) == 0: 
        mentioned_participants = [{
            'text': participant['raw_mention'],
            'entity_id': participant['entity_id'],
            'entity_title': participant['entity_title'],
        } for participant in participants]
        
    mentioned_participants.sort(key = lambda x: len(x['text']))
    # find mention spans
    mentioned_participants = find_keyword_spans(paragraph, mentioned_participants)
    return mentioned_participants
def find_keyword_spans(paragraph, participants):
    for participant in participants:
        keyword_spans = []
        keyword = participant['text']
        pattern = re.compile(re.escape(keyword), re.IGNORECASE)
        matches = pattern.finditer(paragraph)
        for match in matches:
            start_span = match.start()
            end_span = match.end()
            keyword_spans.append((start_span, end_span, keyword))
        participant['spans'] = keyword_spans
    return participants

In [21]:
# paragraph = """
# The article discussed the brief interaction between President Obama and President Rodrigo Duterte at a summit meeting in Laos. After Duterte's profane outburst, Obama canceled their first meeting, but they spoke briefly at a gala dinner. The conversation was not substantive, and they were seated far apart during the dinner.
# """
# find_participant_mentions(["Q76", "Q457786"], paragraph)
article_dict = json.load(open('data/result/AllTheNews/network/articles.json'))
article_participant_span_dict = json.load(open('data/result/AllTheNews/network/server/article_participant_spans.json'))
count = 0
for article_id, article_data in article_dict.items():
    print("{}/{}".format(count, len(article_dict)))
    count += 1
    if len(article_participant_span_dict[article_id]) != 0: continue
    paragraph = article_data['summary']
    participants = [entity for entity in article_data['event']['participants']]
    participant_spans = find_participant_mentions(participants, paragraph)
    article_participant_span_dict[article_id] = participant_spans
    # save_json(article_participant_span_dict, 'data/result/AllTheNews/network/article_participant_spans.json')
save_json(article_participant_span_dict, 'data/result/AllTheNews/network/article_participant_spans.json')

0/7542
1/7542
2/7542
[['the North Pole', Entity(wikidata_entity_id=Q934, wikipedia_entity_title=North Pole), None], ['midwinter', None, 'DATE'], ['Arctic', Entity(wikidata_entity_id=Q25322, wikipedia_entity_title=Arctic), 'GPE'], ['National Science Foundation', Entity(wikidata_entity_id=Q304878, wikipedia_entity_title=National Science Foundation), 'ORG'], ['Pole', Entity not linked to a knowledge base, 'GPE'], ['freezing', None, 'QUANTITY'], ['Canadian', Entity(wikidata_entity_id=Q1196645, wikipedia_entity_title=Canadians), None], ['Nathalie Hasell', Entity not linked to a knowledge base, 'PERSON'], ['the North Pole', Entity(wikidata_entity_id=Q934, wikipedia_entity_title=North Pole), None]]
3/7542
[['Mahon', Entity(wikidata_entity_id=Q7175611, wikipedia_entity_title=Peter Mahon (judge)), 'PERSON'], ['Mahon', Entity(wikidata_entity_id=Q7175611, wikipedia_entity_title=Peter Mahon (judge)), 'PERSON'], ['Labour', Entity(wikidata_entity_id=Q9630, wikipedia_entity_title=Labour Party (UK)), 

In [12]:
article_participant_span_dict = json.load(open('data/result/AllTheNews/network/article_participant_spans.json'))
article_dict = json.load(open('data/result/AllTheNews/network/articles.json'))

In [16]:
for article_id, entities in article_participant_span_dict.items():
    for entity in entities:
        if len(entity['spans']) == 0:
            print(article_id, entity)



103257 {'text': 'Q337084', 'entity_id': 'Q337084', 'entity_title': 'Q337084', 'spans': []}
103722 {'text': 'Q5003490', 'entity_id': 'Q5003490', 'entity_title': 'Q5003490', 'spans': []}
103822 {'text': 'Q7444961', 'entity_id': 'Q7444961', 'entity_title': 'Q7444961', 'spans': []}
104142 {'text': ' [police', 'entity_id': ' [police', 'entity_title': ' [police', 'spans': []}
104144 {'text': 'Q65746426', 'entity_id': 'Q65746426', 'entity_title': 'Q65746426', 'spans': []}
104236 {'text': 'Q403981', 'entity_id': 'Q403981', 'entity_title': 'Q403981', 'spans': []}
104499 {'text': 'Q59154', 'entity_id': 'Q59154', 'entity_title': 'Q59154', 'spans': []}
104923 {'text': 'Q1707206', 'entity_id': 'Q1707206', 'entity_title': 'Q1707206', 'spans': []}
104978 {'text': 'Q7803347', 'entity_id': 'Q7803347', 'entity_title': 'Q7803347', 'spans': []}
105699 {'text': 'Q1157200', 'entity_id': 'Q1157200', 'entity_title': 'Q1157200', 'spans': []}
105869 {'text': 'Q507619', 'entity_id': 'Q507619', 'entity_title': 'Q

In [ ]:
linked_data = json.load(open("data/result/AllTheNews/linked/2016_10p_0819_2.json"))
linked_ids = [article['id'] for article in linked_data]
old_linked_data = json.load(open("data/result/AllTheNews/linked/2016_10p.json"))
old_linked_ids = [article['doc_id'] for article in old_linked_data]
target_data_ids = [old_article_id for old_article_id in old_linked_ids if old_article_id not in linked_ids]
print(len(target_data_ids))

In [ ]:
All_articles = json.load(open("data/raw/AllTheNews/events/2016_10p_0819.json"))
All_articles_dict = { article['id']: article for article in All_articles}
target_data = [All_articles_dict[target_data_id] for target_data_id in target_data_ids]

In [ ]:
new_transformed_data = transform_dataset(target_data)

In [ ]:
for new_data in new_transformed_data:
    linked_data.append(new_data)
save_json(linked_data, 'data/result/AllTheNews/linked/2016_10p_0819_2.json')

In [ ]:
# All the News
# AllTheNews = json.load(open(r'data/result/AllTheNews/preprocessed/2016_10p.json'))
AllTheNews = json.load(open(r'data/raw/AllTheNews/events/2016_10p_0819.json'))
transformed_dataset = transform_dataset(AllTheNews)
print(len(AllTheNews), len(transformed_dataset))
# save_json(transformed_dataset, r'data/result/AllTheNews/linked/2016_10p_0819.json')

In [ ]:
save_json(transformed_dataset, r'data/result/AllTheNews/linked/2016_10p_0819.json')

In [ ]:
prev_dataset = json.load(open(r'data/result/AllTheNews/linked/2016_10p_0819.json'))
for datum in prev_dataset:
    if 'events' in datum.keys():
        event = datum['events']
        datum['event'] = datum['events']
        del datum['events']
save_json(prev_dataset, r'data/result/AllTheNews/linked/2016_10p_0819.json')

In [ ]:
def second_round_linking(events):
    for event in events:
        for argument in event['arguments']:
            if argument['argument_id'] == argument['argument_word']:
                if 'entity_type' in argument.keys(): del argument['entity_type']
                spans = refined.process_text(argument['argument_word'])
                for span in spans:
                    if span.predicted_entity != None and span.predicted_entity.wikidata_entity_id != None:
                        entity_id = span.predicted_entity.wikidata_entity_id
                        entity_title = span.predicted_entity.wikipedia_entity_title
                        argument['argument_id'] = entity_id
                        argument['entity_title'] = entity_title
                        argument['entity_type'] = span.coarse_mention_type
    return events

In [ ]:
from collections import defaultdict
# create node link graph
def construct_network(docs):
    entity_dict = {}
    article_dict = {}
    links = []
    for doc in docs:
        doc_id = doc['id']
        article_dict[doc_id] = doc
        event = doc['event']
        article_id = str(doc_id)
        participants = event['participants']
        # create an entity node for each participant
        for participant in participants:
            participant_id = participant['entity_id']
            participant_word = participant['raw_mention']
            participant_title = participant['entity_title'] 
            participant_entity_type = participant['entity_type'] 

            if participant_id not in entity_dict.keys():
                entity_dict[participant_id] = {
                    "id": participant_id, 
                    "title": participant_title,
                    "entity_type": participant_entity_type,
                    "type": "entity",
                    "mentions": [
                        {
                            "doc_id": doc_id,
                            "mention": participant_word,
                        }
                    ]
                }
            else:
                entity_dict[participant_id]['mentions'].append(
                    {
                        "doc_id": doc_id,
                        "mention": participant_word,
                    }
                )
            participant_ids = [participant['entity_id'] for participant in participants]
            for participant in participants:
                participant_id = participant['entity_id']
                links.append((article_id, participant_id))

    return entity_dict, article_dict, links

def merge_network(dataset):
    entity_dict, article_dict, links = construct_network(dataset)
    B = nx.Graph()
    B.add_nodes_from(list(article_dict.keys()), bipartite=0)
    B.add_nodes_from(list(entity_dict.keys()), bipartite=1)
    B.add_edges_from(links)

    return B, entity_dict, article_dict, links


In [ ]:
transformed_dataset = json.load(open('data/result/AllTheNews/linked/2016_10p_0819.json'))
for datum in transformed_dataset:
    if "events" in datum.keys():
        datum['event'] = datum['events']
        del datum['events']
save_json(transformed_dataset, 'data/result/AllTheNews/linked/2016_10p_0819.json')

In [ ]:
old_linked_data = json.load(open("data/result/AllTheNews/linked/2016_10p.json"))
old_linked_ids = [article['doc_id'] for article in old_linked_data]
res = []
for datum in transformed_dataset:
    if datum["id"] in old_linked_ids:
        res.append(datum)
save_json(res, 'data/result/AllTheNews/linked/2016_10p_0819_2.json')


In [ ]:
# transformed_dataset = json.load(open('data/result/RAMS/gpt_events_dev_linked.json'))
# transformed_dataset = json.load(open('data/result/AllTheNews/linked/2016_10p.json'))
transformed_dataset = json.load(open('data/result/AllTheNews/linked/2016_10p_0819.json'))
B, entity_dict, article_dict, links = merge_network(transformed_dataset)



In [ ]:
print(len(transformed_dataset))

In [ ]:
H = hnx.Hypergraph.from_bipartite(B)
list(H.shape)

In [ ]:
event_hgraph_data = nx.node_link_data(B)
save_json(event_hgraph_data, r'data/result/AllTheNews/network/hgraph.json')
save_json(entity_dict, r'data/result/AllTheNews/network/entities.json')
save_json(article_dict, r'data/result/AllTheNews/network/articles.json')

# save_json(sub_event_hyperedges, r'data/result/AllTheNews/sub_network/sub_event_hyperedges.json')
# save_json(sub_event_links_dict, r'data/result/AllTheNews/sub_network/sub_event_links.json')

In [ ]:
def transform_frontend(nodes, links, nodes_dict, hyper_edges_dict):
    res_nodes = []
    res_links = []
    for node in nodes:
        if node in nodes_dict:
            nodes_dict[node]['type'] = 'entity'
            res_nodes.append(nodes_dict[node])
        else:
            print(hyper_edges_dict[node]['date'])
            date = hyper_edges_dict[node]['date'].replace("-", "/")
            date.replace("-", "/")
            hyper_edges_dict[node]['date'] = date
            hyper_edges_dict[node]['type'] = 'article'
            print(hyper_edges_dict[node]['date'])

            res_nodes.append(hyper_edges_dict[node])
    for link in links:
        source = link[0]
        target = link[1]
        res_links.append({
            "source": source,
            "target": target,
        })
    print(len(res_nodes))
    return {
        "nodes": res_nodes, 
        "links": res_links
    }

In [ ]:
BH = H.bipartite()
network = transform_frontend(list(BH.nodes), list(BH.edges), entity_dict, article_dict)
save_json(network, 'data/result/AllTheNews/network/server/frontend.json')

In [ ]:
network = json.load(open('data/result/AllTheNews/network/server/frontend.json'))
hyperedge_nodes = list(filter(lambda node: node['type'] == 'hyper_edge', network['nodes']))
entity_nodes = list(filter(lambda node: node['type'] == 'entity' and node['id'] != node['title'], network['nodes']))

entity_node_ids = list(map(lambda node: node['id'], entity_nodes))
hyperedge_node_ids = list(map(lambda node: node['id'], hyperedge_nodes))
entity_links = list(filter(lambda link: link['source'] in entity_node_ids or link['target'] in entity_node_ids, network['links']))

In [ ]:
partitions = json.load(open('data/result/AllTheNews/network/server/ravasz_partitions_article.json'))
frontend_data = json.load(open('data/result/AllTheNews/network/server/frontend.json'))
nodes = frontend_data['nodes']
article_nodes = [node for node in nodes if node['type'] == 'hyper_edge']

In [ ]:
old_trigger_doc_id_dict = {}
for article_node in article_nodes:
    old_trigger_doc_id_dict[article_node['id']] = article_node['doc_id']

In [ ]:
old_frontend_data = json.load(open('data/result/AllTheNews/network/server/old/frontend.json'))
old_article_nodes = [node for node in old_frontend_data['nodes'] if node['type'] == 'hyper_edge']
old_article_doc_ids = [node['doc_id'] for node in old_article_nodes]
print(len(old_article_nodes))

In [ ]:
frontend_data = json.load(open('data/result/AllTheNews/network/server/frontend_2.json'))
article_nodes = [node for node in frontend_data['nodes'] if node['type'] == 'article']
entity_nodes = [node for node in frontend_data['nodes'] if node['type'] == 'entity']
print(len(article_nodes), len(entity_nodes))

In [ ]:
articles = { article['id']: article for article in article_nodes }
entities = { entity['id']: entity for entity in entity_nodes }
save_json(articles, 'data/result/AllTheNews/network/articles.json')
save_json(entities, 'data/result/AllTheNews/network/entities.json')

In [ ]:
links = frontend_data['links']
filtered_links = []
for link in links:
    source = link['source']
    target = link['target']
    if (source in articles or source in entities) and (target in articles or target in entities):
        filtered_links.append((source, target))
print(len(filtered_links), len(links))

In [ ]:
B = nx.Graph()
B.add_nodes_from(list(articles.keys()), bipartite=0)
B.add_nodes_from(list(entities.keys()), bipartite=1)
B.add_edges_from(filtered_links)
event_hgraph_data = nx.node_link_data(B)
save_json(event_hgraph_data, r'data/result/AllTheNews/network/hgraph.json')


In [ ]:
B.number_of_nodes(), len(articles), len(entities), len(links)

In [ ]:
for v in B.nodes():
    if v not in articles and v not in entities:
        print(v)